In [88]:
import urllib.request as req
import urllib.parse as par
import datetime
import time
import json
client_id = "2f1UivTJ6JgJILFj0JFp"
client_secret = "UTrYqwSVy2"

In [89]:
def getRequestUrl(url):
    request=req.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    try:
        response=req.urlopen(request)
        if response.getcode()==200:
            print("[%s] Url Request Success" %datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" %(datetime.datetime.now(),url))

In [90]:
def getNaverSearch(node, src_text, start, display):
    base=f"https://openapi.naver.com/v1/search/{node}"
    params=f"?query={par.quote(src_text)}&start={start}&display={display}"
    url=base+params
    resDecode=getRequestUrl(url)
    if resDecode==None:
        return None
    else:
        return json.loads(resDecode)

In [91]:
def getPostData(item, jsonResult, tuple_list, cnt):
    title=item['title'].replace('&quot;','').replace('<b>','').replace('</b>','').replace('&lt;','').replace('&gt;','').replace('&amp;','')
    description=item['description'].replace('&quot;','').replace('<b>','').replace('</b>','').replace('&lt;','').replace('&gt;','').replace('&amp;','')
    link=item['link']
    pDate=datetime.datetime.strptime(item['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate=pDate.strftime('%Y-%m-%d %H:%M:%S')
    tuple_list.append((title,description,link,pDate))
    jsonResult.append({'cnt':cnt,'title':title,'description':description,'link':link,'pDate':pDate})
    #print(title,pDate)

In [92]:
def save_db(tuple_list):
    #db에 저장하기
    import pymysql
    conn=pymysql.connect(host='localhost',user='root',password='1234',db='sampledb',charset='utf8')
    cur=conn.cursor()

    sql='insert into news(title,description,link,pDate) values(%s,%s,%s,%s)'
    cur.executemany(sql,tuple_list)
    conn.commit()
    conn.close()

In [93]:
#def save_csv(tuple_list):
    #csv파일로 저장하기


   

In [94]:
import urllib.parse as par
def main():
    node='news'
    src_text = par.quote(input("검색어를 입력하세요: "))
    cnt=0
    tuple_list=[]
    jsonResult=[]
    jsonResponse=getNaverSearch(node, src_text, 1, 100) #노드, 검색어, start, display
    #print(jsonResponse)
    total=jsonResponse['total']

    while((jsonResponse!=None) and (jsonResponse['display']!=0)):
        for item in jsonResponse['items']:
            cnt+=1
            getPostData(item, jsonResult, tuple_list, cnt)

        start=jsonResponse['start']+jsonResponse['display']
        jsonResponse=getNaverSearch(node, src_text, start, 100)

    print("전체 검색 : %d 건" %cnt)
    #print(jsonResult) json형태로 출력
    with open(f'{src_text}_naver_{node}.json','w',encoding='utf-8') as f:
        jsonFile=json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
        f.write(jsonFile)

In [95]:
main()

[2025-03-25 15:38:04.856781] Url Request Success
[2025-03-25 15:38:05.090716] Url Request Success
[2025-03-25 15:38:05.309526] Url Request Success
[2025-03-25 15:38:05.542557] Url Request Success
[2025-03-25 15:38:05.739838] Url Request Success
[2025-03-25 15:38:05.971401] Url Request Success
[2025-03-25 15:38:06.235176] Url Request Success
[2025-03-25 15:38:06.508449] Url Request Success
[2025-03-25 15:38:06.803710] Url Request Success
[2025-03-25 15:38:07.085007] Url Request Success
HTTP Error 400: Bad Request
[2025-03-25 15:38:07.215287] Error for URL : https://openapi.naver.com/v1/search/news?query=%25EA%25B9%2580%25EC%2597%25B0%25EC%2595%2584&start=1001&display=100
전체 검색 : 1000 건


데이터랩에서 검색어 트렌드
2024.01.01 ~ 2024.12.31
연령별, 성별, 월별 검색어 트렌드를 수집하여, csv에 저장하는 코드를 작성하여 제출

데이터 수집 과제 리스트
- 알리딘 book 데이터 수집(정적크롤링)
- 네이버 이미지 검색(동적 크롤링)
- 스타벅스 매장(동적 크롤링)
- 버스 노선별 버스정유장 도착 예정정보(공공데이터)
- 네이버 데이터랩 검색어 트랜드 가져오기(데이터랩)

In [ ]:
import urllib.parse as par
import csv
#네이버 api인증정보
 
client_id = "huZmgXF9dxPvAANBmCEL"
client_secret = "ku3aqw8UZ2"

#요쳥 url 및 헤더 설정

encText = par.parse.quote("")
url = "https://openapi.naver.com/v1/datalab/search/?query" + encText # JSON 결과
request = par.request.Request(url) #request객체 생성
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = par.request.urlopen(request)
rescode = response.getcode()
item_list=[]

#요청할 기간 및 검색 조건 설정
start_date = "2024-01-01"
end_date = "2024-12-31"
group_names = ["남성_10대", "남성_20대", "남성_30대", "남성_40대", "남성_50대", "여성_10대", "여성_20대", "여성_30대", "여성_40대", "여성_50대"]
ages = ["10", "20", "30", "40", "50"]
genders = ["m", "f"]

#CSV 파일 저장 준비
csv_filename = "naver_trends_2024.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Date", "Gender", "Age", "Keyword", "Ratio"])


#연령별, 성별 검색 트렌드 데이터 요청


